In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import skrf as rf
from skrf import Frequency
from skrf.media import CPW

# Library for computing figures of interest for cpw structures
import ResonatorOptimizer as ro

plt.style.use('seaborn-deep')
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20

In [2]:
# Global Parameters
__er = 11.9
__fo = 7.3e09
__h = 500e-06
__t = 50e-09
__pen_depth = 39e-09
__total_width = 43.8e-06
__length = 8102.64e-06

__keyind = lambda X,X_array: min(enumerate(X_array), key=lambda x: abs(x[1]-X))

# Sweep geometry parameters to find figures of interest for different cpw widths and gaps

In [3]:
# Instantiate parameter sweep object
ps = ro.ParamSweeps(__length,__total_width,__fo,__er,__h,__t,__pen_depth)

# Sweep geometry to see effect on parameters of interest
minw, maxw, wit = [2, (__total_width*1e06)-1, 0.01]
# minw, maxw, wit = [2, 8, 0.01]
params = ps.width_to_gap(minw,maxw,wit=wit)
# ps.plot_params(params)

In [4]:
# opt = ro.Optimizer()
# opt.set_geometry(params,'Lkl',2.190023e+09,wavelength=0.5)



# cav_ind,cav_val = __keyind(50,params.Zki)
# cav_width = params.width[cav_ind]
# cav_gap = params.gap[cav_ind]
# cav_length = 8102.64e-06

# cavity = ro.cpwCalcs(cav_width,cav_gap,cav_length,__fo, __er, h=__h, t=__t, pen_depth=__pen_depth)
# cav_f0 = 1 / (2 *cav_length* np.sqrt(cavity.capacitance_per_length() * cavity.Ltotal()))
# cav_vp = 1 / (np.sqrt(cavity.capacitance_per_length() * cavity.Ltotal()))
# cav_E = .5*(cavity.phase_velocity() / cavity.resonant_freq())

# cav_f0*1e-09
# cav_vp*1e-06
# cav_E*1e06

In [5]:
def set_geometry(swept_params,param,param_val,wavelength=1):
    """ Sets the width, gap, and length of the cpw based on a desired input parameter"""
    parameter = swept_params[param]
    ind, val = __keyind(param_val,parameter)
    
    width = swept_params.width[ind]
    gap = swept_params.gap[ind]
    
    dic = {'width':width,'gap':gap}
    df = pd.DataFrame(data=dic,index=[0])

    return df

In [11]:
def get_fo(swept_params,param,param_val,wavelength=1):
    geo = set_geometry(swept_params,param,param_val,wavelength=wavelength)
    index = np.where(params.width==__keyind(params.width, geo.width.values)[1])[0][0]
    fo = params.res_freq[index]
    return geo, index, fo

def calc_length(swept_params,fo=1e09,Lkl=0,wavelength=1):
    geo1, index1, fo1 = get_fo(swept_params,'res_freq',fo,wavelength=0.5)
    geo2, index2, fo2 = get_fo(swept_params,'Lkl',Lkl,wavelength=0.5)                              
    
    l1 = .5*(swept_params.vp[index2] /swept_params.res_freq[index2])
    l2 = .5*( 1 / (2 * (wavelength) * fo1 * np.sqrt(swept_params.Cl[index2] * swept_params.Ltot[index2])) )
    
    df = pd.DataFrame({'lo':[l1], 'leff':l2})
    return df

In [15]:
cav_length = calc_length(params, fo=7.1943e+09, Lkl=2e-06, wavelength=0.5)

In [16]:
# Get desired resonant frequency
geo = set_geometry(params,'res_freq',7.1943e+09,wavelength=0.5)
index = np.where(params.width==__keyind(params.width, geo.width.values)[1])[0][0]
fo = params.res_freq[index]

# Get acquired resonant frequency with current geometry
geo_new = set_geometry(params,'Lkl',2e-06,wavelength=0.5)
index_new = np.where(params.width==__keyind(params.width, geo_new.width.values)[1])[0][0]
fo_new = params.res_freq[index_new]

cal_length = params.vp[index_new] / params.res_freq[index_new]
new_length = 1 / (2 * (0.5) * fo * np.sqrt(params.Cl[index_new] * params.Ltot[index_new]))
length = .5*new_length

# Create Cavity object

In [17]:
cav_width, cav_gap = set_geometry(params,'Lkl',2e-06,wavelength=0.5)
cav_length = calc_length(params, fo=7.1943e+09, Lkl=2e-06, wavelength=0.5)
cavity = ro.cpwCalcs(geo_new.width,geo_new.gap,cav_length,__fo, __er, h=__h, t=__t, pen_depth=__pen_depth)
cav_f0 = 1 / (2 *cav_length.leff* np.sqrt(cavity.capacitance_per_length() * cavity.Ltotal()))
cav_f0*1e09

0    7.194337e+18
dtype: float64

# Create LowZ object

In [18]:
lowZ_geo = set_geometry(params,'Zki',30,wavelength=0.25)
lowZ_width, lowZ_gap = [lowZ_geo.width.values[0], lowZ_geo.gap.values[0]]
lowZ_length = calc_length(params, fo=7.1943e+09, Lkl=2e-07, wavelength=0.25)

lowZ = ro.cpwCalcs(lowZ_width,lowZ_gap,lowZ_length,__fo, __er, h=__h, t=__t, pen_depth=__pen_depth)
low_f0 = 1 / (4 * lowZ_length.leff* np.sqrt(lowZ.capacitance_per_length() * lowZ.Ltotal() ) )
low_f0*1e-09

0    2.049393
Name: leff, dtype: float64

In [24]:
lowZ_geo.width

0    0.000037
Name: width, dtype: float64